In [16]:
import open3d as o3d
import numpy as np

# Point Cloud 파일을 읽어오는 함수
def load_point_cloud(file_path):
    pcd = o3d.io.read_point_cloud(file_path)
    return pcd

# Point Cloud를 메쉬로 변환하는 함수
def point_cloud_to_mesh(pcd):
    # Point Cloud를 voxel downsample
    voxel_size = 0.005
    pcd_down = pcd.voxel_down_sample(voxel_size)

    # 법선 계산
    radius_normal = voxel_size * 2
    pcd_down.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))

    # 메쉬 생성 (Ball Pivoting Algorithm)
    distances = pcd_down.compute_nearest_neighbor_distance()
    avg_dist = np.mean(distances)
    radius = 3 * avg_dist
    mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(
        pcd_down, o3d.utility.DoubleVector([radius, radius * 2])
    )

    # 색상 추가
    if pcd.has_colors():
        vertex_colors = np.asarray(pcd_down.colors)
        mesh.vertex_colors = o3d.utility.Vector3dVector(vertex_colors)


    return mesh

# 메쉬를 저장하는 함수
def save_mesh_to_drive(mesh, file_path):
    o3d.io.write_triangle_mesh(file_path, mesh)



In [17]:
import plotly.graph_objs as go
import plotly.io as pio
import open3d as o3d
import numpy as np

def visualize_point_cloud_with_plotly(pcd, html_file):
    # 포인트 클라우드의 좌표와 색상 정보를 추출
    points = np.asarray(pcd.points)
    print(f"Number of points: {points.shape[0]}")
    
    if points.shape[0] == 0:
        print("Error: The point cloud has no points.")
        return
    
    # if pcd.has_colors():
    #     colors = np.asarray(pcd.colors)
    #     rgb_colors = ['rgb({},{},{})'.format(int(r*255), int(g*255), int(b*255)) for r, g, b in colors]
    # else:
    rgb_colors = ['rgb(0,0,255)'] * points.shape[0]  # 기본 색상을 파란색으로 설정

    x, y, z = points.T

    # x, y, z 범위 확인
    print(f"x range: {x.min()} to {x.max()}")
    print(f"y range: {y.min()} to {y.max()}")
    print(f"z range: {z.min()} to {z.max()}")

    # Plotly의 Scatter3d 객체 생성
    point_cloud_plot = go.Scatter3d(
        x=x, y=y, z=z,
        mode='markers',
        marker=dict(
            size=2,
            color=rgb_colors,
            opacity=0.8
        )
    )

    # 레이아웃 설정
    layout = go.Layout(
        scene=dict(
            xaxis=dict(visible=False),
            yaxis=dict(visible=False),
            zaxis=dict(visible=False)
        ),
        margin=dict(r=0, l=0, b=0, t=0)  # 여백 제거
    )

    # Figure 생성 및 HTML 파일로 저장
    fig = go.Figure(data=[point_cloud_plot], layout=layout)
    pio.write_html(fig, file=html_file, auto_open=True)
    pio.show(fig)

def visualize_mesh_with_plotly(mesh, html_file):
    vertices = np.asarray(mesh.vertices)
    triangles = np.asarray(mesh.triangles)
    colors = np.asarray(mesh.vertex_colors) if mesh.has_vertex_colors() else None

    x, y, z = vertices.T
    i, j, k = triangles.T

    if colors is not None:
        vertexcolor = ['rgb({},{},{})'.format(int(r * 255), int(g * 255), int(b * 255)) for r, g, b in colors]
    else:
        vertexcolor = 'lightblue'

    mesh_plot = go.Mesh3d(
        x=x, y=y, z=z,
        i=i, j=j, k=k,
        vertexcolor=vertexcolor,
        opacity=0.50
    )

    layout = go.Layout(
        scene=dict(
            xaxis=dict(visible=False),
            yaxis=dict(visible=False),
            zaxis=dict(visible=False)
        )
    )

    fig = go.Figure(data=[mesh_plot], layout=layout)
    pio.write_html(fig, file=html_file, auto_open=True)
    pio.show(fig)

In [18]:
# Voxel화 함수
def voxelize_point_cloud(pc, voxel_size=0.01):
    voxel_grid = o3d.geometry.VoxelGrid.create_from_point_cloud(pc, voxel_size=voxel_size)
    return voxel_grid

# VoxelGrid를 Point Cloud로 변환하는 함수
def voxel_grid_to_point_cloud(voxel_grid):
    points = []
    colors = []
    for voxel in voxel_grid.get_voxels():
        points.append(voxel.grid_index * voxel_grid.voxel_size)
        colors.append(voxel.color)

    pc = o3d.geometry.PointCloud()
    pc.points = o3d.utility.Vector3dVector(points)
    pc.colors = o3d.utility.Vector3dVector(colors)
    return pc

#Voxel Point Cloud Visualization
def visualize_voxel_pc_with_plotly(voxel_grid):
    points = []
    colors = []
    for voxel in voxel_grid.get_voxels():
        points.append(voxel.grid_index * voxel_grid.voxel_size)
        colors.append(voxel.color)
    
    points = np.array(points)
    colors = np.array(colors)

    x, y, z = points.T
    rgb_colors = ['rgb({},{},{})'.format(int(r*255), int(g*255), int(b*255)) for r, g, b in colors]
    
    voxel_plot = go.Scatter3d(
        x=x, y=y, z=z,
        mode='markers',
        marker=dict(
            size=5,
            color=rgb_colors,
            opacity=0.8
        )
    )

    layout = go.Layout(
        scene=dict(
            xaxis=dict(visible=False),
            yaxis=dict(visible=False),
            zaxis=dict(visible=False)
        )
    )

    fig = go.Figure(data=[voxel_plot], layout=layout)
    pio.show(fig)

# Voxel Visualization
def visualize_voxel_grid_with_plotly(voxel_grid, html_file):
    x = []
    y = []
    z = []
    i = []
    j = []
    k = []
    vertex_colors = []
    cube_vertices = np.array([[0, 0, 0], [1, 0, 0], [1, 1, 0], [0, 1, 0],
                              [0, 0, 1], [1, 0, 1], [1, 1, 1], [0, 1, 1]])
    cube_faces = np.array([[0, 1, 2], [0, 2, 3],
                           [4, 5, 6], [4, 6, 7],
                           [0, 1, 5], [0, 4, 5],
                           [1, 2, 6], [1, 5, 6],
                           [2, 3, 7], [2, 6, 7],
                           [3, 0, 4], [3, 4, 7]])
    for idx, voxel in enumerate(voxel_grid.get_voxels()):
        voxel_center = voxel.grid_index * voxel_grid.voxel_size
        voxel_vertices = voxel_center + cube_vertices * voxel_grid.voxel_size
        x.extend(voxel_vertices[:, 0])
        y.extend(voxel_vertices[:, 1])
        z.extend(voxel_vertices[:, 2])
        voxel_color = voxel.color
        vertex_colors.extend([voxel_color for _ in range(8)])
        i.extend(cube_faces[:, 0] + idx * 8)
        j.extend(cube_faces[:, 1] + idx * 8)
        k.extend(cube_faces[:, 2] + idx * 8)

    vertex_colors = ['rgb({},{},{})'.format(int(r * 255), int(g * 255), int(b * 255)) for r, g, b in vertex_colors]

    voxel_plot = go.Mesh3d(
        x=x, y=y, z=z,
        i=i, j=j, k=k,
        vertexcolor=vertex_colors,
        opacity=0.8
    )

    layout = go.Layout(
        scene=dict(
            xaxis=dict(visible=False),
            yaxis=dict(visible=False),
            zaxis=dict(visible=(False))
    ))

    fig = go.Figure(data=[voxel_plot], layout=layout)
    pio.write_html(fig, file=html_file, auto_open=True)
    pio.show(fig)


In [23]:
import os
ex_path = 'C:/Users/soyso/Downloads/dataset/mouse/mouse.ply'
out_path = 'C:/Users/soyso/Downloads/dataset/mouse/mouse_mesh_color.ply'

pcd = load_point_cloud(ex_path)
print(pcd)
mesh = point_cloud_to_mesh(pcd)
print(mesh)
output = os.path.join(out_path)
save_mesh_to_drive(mesh, output)
print(f"save mesh at {output}")

PointCloud with 1007157 points.
TriangleMesh with 6894 points and 9950 triangles.
save mesh at C:/Users/soyso/Downloads/dataset/mouse/mouse_mesh_color.ply


In [ ]:
# 저장된 메쉬를 다시 읽어오기
out_path = 'C:/Users/soyso/Downloads/dataset/gate_mesh.ply'
loaded_mesh = o3d.io.read_triangle_mesh(out_path)
print(loaded_mesh)

# 저장된 메쉬 시각화
visualize_mesh_with_plotly(loaded_mesh)

In [ ]:
# Voxel화
voxel_grid = voxelize_point_cloud(pcd)

# VoxelGrid를 Point Cloud로 변환
voxel_pc = voxel_grid_to_point_cloud(voxel_grid)

# Voxel Point Cloud 저장
o3d.io.write_point_cloud("/content/drive/MyDrive/data/voxel_point_cloud.ply", voxel_pc)

In [ ]:
# Voxel Point Cloud
visualize_voxel_pc_with_plotly(voxel_grid)

# Assuming `voxel_grid` is your VoxelGrid object
visualize_voxel_grid_with_plotly(voxel_grid, "/content/drive/MyDrive/data/mesh_visualization.html")

In [4]:
import os
from plyfile import PlyData

def format_size(size_in_bytes):
    # 바이트를 KB 및 MB로 변환하여 출력 형식을 지정합니다.
    kb_size = size_in_bytes / 1024
    mb_size = kb_size / 1024
    if mb_size >= 1:
        return f"{mb_size:.2f} MB"
    else:
        return f"{kb_size:.2f} KB"

def print_ply_file_size(file_path):
    try:
        # 파일 크기 출력 (바이트 단위)
        file_size = os.path.getsize(file_path)
        formatted_size = format_size(file_size)
        print(f"파일 크기: {formatted_size}")

        # PLY 파일 데이터 읽기
        ply_data = PlyData.read(file_path)
        
        # PLY 파일의 요소들 출력
        # for element in ply_data.elements:
        #     print(f"요소 이름: {element.name}")
        #     print(f"요소 개수: {element.count}")
        #     for property in element.properties:
        #         print(f"  속성 이름: {property.name}, 속성 타입: {property.dtype}")

    except Exception as e:
        print(f"파일을 읽는 중 오류가 발생했습니다: {e}")

# 예제 파일 경로
pc_path = 'C:/Users/soyso/Downloads/min.ply'  # 여기에 실제 PLY 파일 경로를 입력하세요

print_ply_file_size(pc_path)

mesh_path = 'C:/Users/soyso/Downloads/min_mesh.ply'

print_ply_file_size(mesh_path)

파일 크기: 63.42 MB
파일 크기: 3.61 MB
